In [1]:
import os

In [2]:
%pwd

'c:\\Users\\ranji\\My Projects\\Projects\\Kidney-Disease-Classification-Deep-Learning-MLOps-Pipeline\\research'

In [3]:
os.chdir("../")

In [4]:
%pwd

'c:\\Users\\ranji\\My Projects\\Projects\\Kidney-Disease-Classification-Deep-Learning-MLOps-Pipeline'

In [9]:
from dataclasses import dataclass
from pathlib import Path


@dataclass(frozen=True)
class DataIngestionConfig:
    root_dir: Path
    source_URL: str
    local_data_file: Path
    unzip_dir: Path

In [10]:
from src.KidneyDiseaseClassifier.constants import *
from src.KidneyDiseaseClassifier.utils.common import read_yaml, create_directories

In [11]:
class ConfigurationManager:
    def __init__(
        self,
        config_filepath = CONFIG_FILE_PATH,
        params_filepath = PARAMS_FILE_PATH):

        self.config = read_yaml(config_filepath)
        self.params = read_yaml(params_filepath)

        create_directories([self.config.artifacts_root])


    
    def get_data_ingestion_config(self) -> DataIngestionConfig:
        config = self.config.data_ingestion

        create_directories([config.root_dir])

        data_ingestion_config = DataIngestionConfig(
            root_dir=config.root_dir,
            source_URL=config.source_URL,
            local_data_file=config.local_data_file,
            unzip_dir=config.unzip_dir 
        )

        return data_ingestion_config

In [12]:
import os
import zipfile
import boto3
from urllib.parse import urlparse
from src.KidneyDiseaseClassifier import logger
from src.KidneyDiseaseClassifier.utils.common import get_size

In [13]:
class DataIngestion:
    def __init__(self, config: DataIngestionConfig):
        self.config = config

    def download_file(self) -> str:
        '''
        Downloads ZIP file from an S3 public URL and saves it locally.
        '''
        try:
            dataset_url = self.config.source_URL
            zip_download_path = self.config.local_data_file
            os.makedirs(os.path.dirname(zip_download_path), exist_ok=True)

            logger.info(f"Downloading data from {dataset_url} into {zip_download_path}")

            # Parse bucket and key from the S3 URL
            parsed_url = urlparse(dataset_url)
            bucket_name = parsed_url.netloc.split('.')[0]  
            object_key = parsed_url.path.lstrip('/')       

            s3 = boto3.client('s3')
            with open(zip_download_path, 'wb') as f:
                s3.download_fileobj(bucket_name, object_key, f)

            logger.info(f"Downloaded data from {dataset_url} into file {zip_download_path}")
            return zip_download_path

        except Exception as e:
            logger.error(f"Failed to download file from S3: {e}")
            raise e

    def extract_zip_file(self):
        """
        Extracts the downloaded ZIP file into the target directory.
        """
        try:
            unzip_path = self.config.unzip_dir
            zip_path = self.config.local_data_file

            os.makedirs(unzip_path, exist_ok=True)
            logger.info(f"Extracting {zip_path} to {unzip_path}")

            with zipfile.ZipFile(zip_path, 'r') as zip_ref:
                zip_ref.extractall(unzip_path)

            logger.info(f"Extraction completed to folder: {unzip_path}")

        except Exception as e:
            logger.error(f"Failed to extract zip file: {e}")
            raise e



In [14]:
try:
    config = ConfigurationManager()
    data_ingestion_config = config.get_data_ingestion_config()
    data_ingestion = DataIngestion(config=data_ingestion_config)
    data_ingestion.download_file()
    data_ingestion.extract_zip_file()
except Exception as e:
    raise e

[2025-07-10 14:32:44,573: INFO: common: yaml file: config\config.yaml loaded successfully]
[2025-07-10 14:32:44,573: INFO: common: yaml file: params.yaml loaded successfully]
[2025-07-10 14:32:44,581: INFO: common: created directory at: artifacts]
[2025-07-10 14:32:44,581: INFO: common: created directory at: artifacts/data_ingestion]
[2025-07-10 14:32:44,581: INFO: 2186131085: Downloading data from https://ct-kidney-data.s3.us-east-2.amazonaws.com/ct-kidney-images-data.zip into artifacts/data_ingestion/data.zip]
[2025-07-10 14:32:44,606: INFO: credentials: Found credentials in shared credentials file: ~/.aws/credentials]
[2025-07-10 14:32:53,321: INFO: 2186131085: Downloaded data from https://ct-kidney-data.s3.us-east-2.amazonaws.com/ct-kidney-images-data.zip into file artifacts/data_ingestion/data.zip]
[2025-07-10 14:32:53,327: INFO: 2186131085: Extracting artifacts/data_ingestion/data.zip to artifacts/data_ingestion]
[2025-07-10 14:32:57,020: INFO: 2186131085: Extraction completed to